In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
import os
import pandas as pd

In [ ]:
class TicTacToeDataset(Dataset):
    def __init__(self):
        self.img_dir = 'train'

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.seq = nn.Sequential(
            self._block(1, 18, 3),
            self._block(18, 36, 3),
            self._block(36, 48, 3),
            self._block(48, 64, 3),
            self._block(64, 80, 3),
            self._block(80, 96, 3),
            self._block(96, 112, 3),
            self._block(112, 128, 3),
            self._block(128, 144, 3),
            self._block(144, 156, 3),
            Flatten(),
            nn.Linear(64*156, 3, bias=False),
            nn.BatchNorm1d(3),
        )

    def _block(self, input_dim, output_dim, kernel_size):
        return nn.Sequential(
            nn.Conv2d(input_dim, output_dim, kernel_size, bias=False),
            nn.BatchNorm2d(output_dim),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.seq(x)
        return F.log_softmax(x, dim=1)

class Flatten(nn.Module):
    def forward(self, x):
        return torch.flatten(x.permute(0, 2, 3, 1), 1)


In [ ]:
model = Model()
optimizer = Adam(model.parameters(), lr=0.001)

dataset = TicTacToeDataset()
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [ ]:
for (x, y) in dataloader:
    y_pred = model(x)
    loss = F.nll_loss(y_pred, y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()